In [1]:
! pip install -q cassio datasets langchain langchain-huggingface langchain-groq langchain_community

In [2]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
from datasets import load_dataset
import cassio

In [4]:
!pip install pyPDF2

In [5]:
from PyPDF2 import PdfReader

In [ ]:
GROQ_API_KEY=""
ASTRA_DB_APPLICATION_TOKEN=""
ASTRA_DB_ID = ""

In [7]:
pdfreader = PdfReader("kep.pdf")

In [8]:
from typing_extensions import Concatenate
raw_text = ""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text+=content

In [9]:
raw_text

' Architectur al HardwareKEPT O is a brand that is widely used in mid-le vel residen \x00al and commer cial mark ets.\nWe blend cu\x00ng-edg e technology with contempor ary design. of products in our collec\x00on,\nour best-selling items include Kitchen hardware and accessories combined with close t slides,\nsliding par\x00\x00ons, furnitur e har dware, and bed ﬁ\x00ngs. 14 y ears of e xperience in the har dware\nmark et, which has turned into a passion for the Kitchen segmen t. I had always wanted to\nlaunch my own kitchen ﬁ\x00ngs brand, KEPT O, and a\x00er years of hard work and knowledg e\nacquisi\x00on, I w as able t o do it.\nGOALS & MISSION \nTO PR OVIDE SER VICE S AND PR ODUC TS OF HIGH QU ALITY A T RE ASONABLE PRICE S\nFOR ALL C ONSUMER SE GMENT .About  Architectur al Hardware\nINDEX\n1. Kit chen Accessories\n2. Tandem Bo x\n3. Aut o Hing es\n4. Pr oﬁles \n5. Telesc opic Channel\n6. Codr o Channel\n7. Sliding S ystem\n8. W ardrobe Accessories\n9. Bed Fi\x00ngs\n10. Door Closer

In [10]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [ ]:
import os
os.environ["GROQ_API_KEY"] = ""

In [12]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)

In [14]:

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap = 200,
    length_function= len,
)
texts = text_splitter.split_text(raw_text)

In [15]:
texts[:50]

['Architectur al HardwareKEPT O is a brand that is widely used in mid-le vel residen \x00al and commer cial mark ets.\nWe blend cu\x00ng-edg e technology with contempor ary design. of products in our collec\x00on,\nour best-selling items include Kitchen hardware and accessories combined with close t slides,\nsliding par\x00\x00ons, furnitur e har dware, and bed ﬁ\x00ngs. 14 y ears of e xperience in the har dware\nmark et, which has turned into a passion for the Kitchen segmen t. I had always wanted to\nlaunch my own kitchen ﬁ\x00ngs brand, KEPT O, and a\x00er years of hard work and knowledg e\nacquisi\x00on, I w as able t o do it.\nGOALS & MISSION \nTO PR OVIDE SER VICE S AND PR ODUC TS OF HIGH QU ALITY A T RE ASONABLE PRICE S\nFOR ALL C ONSUMER SE GMENT .About  Architectur al Hardware\nINDEX\n1. Kit chen Accessories',
 "GOALS & MISSION \nTO PR OVIDE SER VICE S AND PR ODUC TS OF HIGH QU ALITY A T RE ASONABLE PRICE S\nFOR ALL C ONSUMER SE GMENT .About  Architectur al Hardware\nINDEX\n1.

In [16]:
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 30 headlines.


In [17]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): what is this pdf all about?

QUESTION: "what is this pdf all about?"


ANSWER: "This text appears to be a portion of a research paper or technical document about the Transformer model, a type of neural network architecture used in natural language processing tasks, such as machine translation. The text discusses the architecture of the Transformer model, including its encoder and decoder stacks, self-attention mechanisms, and hyperparameters. It also presents experimental results and compares the performance of different model configurations."

FIRST DOCUMENTS BY RELEVANCE:


    [0.5868] "of continuous representations z= (z1, ..., z n). Given z, the decoder then generates ..."
    [0.5651] "we found it beneficial to linearly project the queries, keys and values htimes with  ..."
    [0.5606] "in the distance between positions, linearly for ConvS2S and logarithmically for Byte ..."
    [0.5548] "model. All metrics are on the English-to-German translation development set, newstes ..."

What's your next question (or type 'quit' to exit): what is the mrp of tandem pantry

QUESTION: "what is the mrp of tandem pantry"


ANSWER: "The MRP of Tandem Pantry is:

- 35999 for 600mm
- 38999 for (no size specified, but it is mentioned as an alternative to the 600mm option, possibly for 450mm)"

FIRST DOCUMENTS BY RELEVANCE:


    [0.6780] "600mm35999
38999So  Close
So  Close
So  Close
So  Close
So  Close
Tandem P antry
Uni ..."
    [0.6550] "GOALS & MISSION 
TO PR OVIDE SER VICE S AND PR ODUC TS OF HIGH QU ALITY A T RE ASONA ..."
    [0.6442] "E- Swing with tw o tray &  It's a high
Gray pla ted  wir e structur e and
melamine C ..."
    [0.6271] "9999
10999
11999Bo el Pull out with Glass bask et
Two  re Mul  Slider Side
Moun  ng  ..."

What's your next question (or type 'quit' to exit): exit

QUESTION: "exit"


ANSWER: "Goodbye."

FIRST DOCUMENTS BY RELEVANCE:


    [0.6206] "K SLIDE 12 SC/ZK
K SLIDE 14 SC/ZK
K SLIDE 16 SC/ZK
K SLIDE 18 SC/ZK
K SLIDE 20 SC/ZK ..."
    [0.5847] "9999
10999
11999Bo el Pull out with Glass bask et
Two  re Mul  Slider Side
Moun  ng  ..."
    [0.5798] "layer in a typical sequence transduction encoder or decoder. Motivating our use of s ..."
    [0.5740] "600mm35999
38999So  Close
So  Close
So  Close
So  Close
So  Close
Tandem P antry
Uni ..."

What's your next question (or type 'quit' to exit): quit
